In [33]:
import numpy as np
import math
from numpy import linalg as LA

6. Система координат камеры получается из мировой поворотом на <code>45 градусов вокруг оси z и трансляцией на 10 вдоль оси z</code>. Внутренние параметры камеры: <code>fx=fy=400, cx=960, cy=540</code>. Написать программу, которая выводит на экран матрицу проекции камеры и целочисленные координаты пикселя на изображении, соответствующего трехмерной точке с координатами <code>(10, -10, 100)</code> в мировой системе координат.

In [34]:
r = np.around(math.cos(math.radians(45)),3)

rt = np.array([[math.cos(math.radians(45)), -math.sin(math.radians(45)), 0, 0],
              [math.sin(math.radians(45)), math.cos(math.radians(45)), 0, 0],
              [0, 0, 1, 10]], dtype=np.float64)

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]], dtype = np.float64)

point = np.array([10, -10, 100, 1], dtype = np.float64)

p = k @ rt

n_point = p @ point

p = np.around(p, 2)

n_point = np.around(n_point)

print("Матрица проекции камеры:\n")
p


Матрица проекции камеры:



array([[ 2.8284e+02, -2.8284e+02,  9.6000e+02,  9.6000e+03],
       [ 2.8284e+02,  2.8284e+02,  5.4000e+02,  5.4000e+03],
       [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+01]])

In [35]:
print("Координаты пикселя: \n")

print("x = " + str(n_point[0]/n_point[2]))
print("y = " + str(n_point[1]/n_point[2]))

Координаты пикселя: 

x = 1011.4272727272727
y = 540.0


7. Используя прямое линейное преобразование (<code>DLT</code>) и <code>SVD</code>, написать программу, которая принимает на вход множество пар двумерных точек, и выводит на экран соответствующую этим парам матрицу гомографии. Написать генератор пар двумерных точек для теста алгоритма, удовлетворяющих заданному заранее преобразованию гомографии.

1.009090909090909

8. Модифицировать генератор из <code>Задачи 7</code> так, чтобы он выдавал <code>n=1000</code> соответствий, из которых <code>30%</code> соответствуют заданному заранее преобразованию гомографии, а остальные взяты случайно (преобразованию гомографии не удовлетворяют). Реализовать нахождение преобразования гомографии, используя <code>DLT</code> и <code>RANSAC</code>.